In [1]:
#20l-0978
#20l-1034
#20l-1096

# Part1
import pandas as pd
import numpy as np
df=pd.read_csv('/content/Training.csv', header=0)

#Insert 2 Columns Female/Male, Drop Gender column
df.insert(2,'Male',None)
df.insert(2,'Female',None)
# df.insert(len(df.columns),'Numerical',None)
df['Female']=df['Gender'].map({1:0,2:1})
df['Male']=df['Gender'].map({1:1,2:0})
df=df.drop(['Gender'],axis=1)


#Insert 0 in place of Null Values
df=df.fillna(0)

#Drop rows if prognosis is null
df=df.dropna(subset='prognosis')
DiseaseLst=df['prognosis'].unique()
count=len(DiseaseLst)
print(count, "Diseases to be Diagnosed")

DiseaseLst=DiseaseLst.tolist()
print(DiseaseLst)
# o=0;
# df['Numerical']=df['prognosis'].map({DiseaseLst[o]:o})
# o+=1
# df['Numerical']=df['prognosis'].map({DiseaseLst[o]:o})


#Load Precautions and find unique precautions to insert new cols
prec=pd.read_csv('/content/symptom_precaution.csv', header=0)

# here//////////////////////////////////////////////////////////////////////////////////////
lst=[]
lst=pd.unique(prec[['Precaution_1','Precaution_2','Precaution_3','Precaution_4']].values.ravel())
for each in lst:
  prec.insert(len(prec.columns),each,None)

#One hot encoding for precautions
j=0
for i in prec['Precaution_1']:
  for each in lst:
    if i==each:
      prec[i][j]=1
  j+=1
j=0
for i in prec['Precaution_2']:
  for each in lst:
    if i==each:
      prec[i][j]=1
  j+=1
j=0
for i in prec['Precaution_3']:
  for each in lst:
    if i==each:
      prec[i][j]=1
  j+=1
j=0
for i in prec['Precaution_4']:
  for each in lst:
    if i==each:
      prec[i][j]=1
  j+=1

#Fill Null Cells with 0
prec=prec.fillna(0)

prec=prec.drop(['Precaution_1','Precaution_2','Precaution_3','Precaution_4'],axis=1)
# Till here/////////////////////////////////////////////////////////


#Merge the two on prognosis
# prec=prec.rename(columns={'Disease':'prognosis'})
# df=pd.merge(df,prec, on='prognosis')


#Save csv file
df=df.drop(['PatientID','Age','Male','Female'],axis=1)
print(df)
df=df.to_csv('DataSet.csv')


# Part2
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the dataset
data = pd.read_csv('/content/DataSet.csv') 
data=data.drop(['Unnamed: 0'],axis=1)
# Split the dataset into features and target variable
X = data.drop('prognosis', axis=1)
y = data['prognosis']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
lr_model = LogisticRegression(max_iter=1000)

lr_model.fit(X_train, y_train)

# Train a decision tree classifier model
dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

# Make predictions on the testing set
lr_preds = lr_model.predict(X_test)
dt_preds = dt_model.predict(X_test)

# Evaluate the performance of the logistic regression model
lr_accuracy = accuracy_score(y_test, lr_preds)
lr_precision = precision_score(y_test, lr_preds,average='micro')
lr_recall = recall_score(y_test, lr_preds,average='micro')
lr_f1_score = f1_score(y_test, lr_preds,average='micro')

# Evaluate the performance of the decision tree classifier model
dt_accuracy = accuracy_score(y_test, dt_preds)
dt_precision = precision_score(y_test, dt_preds,average='micro')
dt_recall = recall_score(y_test, dt_preds,average='micro')
dt_f1_score = f1_score(y_test, dt_preds,average='micro')

# Print evaluation metrics for both models
print('Logistic Regression:')
print('Accuracy:', lr_accuracy)
print('Precision:', lr_precision)
print('Recall:', lr_recall)
print('F1 Score:', lr_f1_score)

print('\nDecision Tree Classifier:')
print('Accuracy:', dt_accuracy)
print('Precision:', dt_precision)
print('Recall:', dt_recall)
print('F1 Score:', dt_f1_score)

# Save the best performing model (in this example, the one with highest accuracy)
if lr_accuracy > dt_accuracy:
    best_model = lr_model
    best_model_name = 'Logistic Regression'
else:
    best_model = dt_model
    best_model_name = 'Decision Tree Classifier'

# Save the best model for further processing
import joblib
joblib.dump(best_model, 'best_model.pkl')  # Replace with the desired filename for the saved model
print(f'\n{best_model_name} saved as best_model.pkl')

model = joblib.load('best_model.pkl')




# part 3


41 Diseases to be Diagnosed
['Fungal infection', 'Allergy', 'GERD', 'Chronic cholestasis', 'Drug Reaction', 'Peptic ulcer diseae', 'AIDS', 'Diabetes ', 'Gastroenteritis', 'Bronchial Asthma', 'Hypertension ', 'Migraine', 'Cervical spondylosis', 'Paralysis (brain hemorrhage)', 'Jaundice', 'Malaria', 'Chicken pox', 'Dengue', 'Typhoid', 'hepatitis A', 'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E', 'Alcoholic hepatitis', 'Tuberculosis', 'Common Cold', 'Pneumonia', 'Dimorphic hemmorhoids(piles)', 'Heart attack', 'Varicose veins', 'Hypothyroidism', 'Hyperthyroidism', 'Hypoglycemia', 'Osteoarthristis', 'Arthritis', '(vertigo) Paroymsal  Positional Vertigo', 'Acne', 'Urinary tract infection', 'Psoriasis', 'Impetigo']
      itching  skin_rash  nodal_skin_eruptions  continuous_sneezing  \
0           1          1                     1                    0   
1           0          1                     1                    0   
2           1          0                     1          

In [7]:
%%writefile app.py
import pandas as pd
import streamlit as st
import joblib

# Load the saved model
model = joblib.load('best_model.pkl')  # Replace with the path to your saved model

# Load the dataset
data = pd.read_csv('/content/DataSet.csv')
data=data.drop(['Unnamed: 0'],axis=1)
# Set up the app layout
st.set_page_config(page_title='Medical Diagnosis', page_icon=':microscope:')
st.title('Medical Diagnosis')
st.write('Enter the patient symptoms below and click the "Diagnose" button to get a diagnosis.')

# Create input fields for symptoms
input_cols = data.columns[:-1]
input_vals = []
for col in input_cols:
    input_val = st.radio(col, ('Yes','No'))
    if input_val=='Yes':
      input_val=1
    else:
      input_val=0
    input_vals.append(input_val)

# Create a button to diagnose the patient
if st.button('Diagnose'):
    # Create a DataFrame from the input values
    input_data = pd.DataFrame([input_vals], columns=input_cols)


    # Make a prediction using the model
    prediction = model.predict(input_data)
    # Print the diagnosis
    st.write(f'Diagnosis: {prediction[0]}') 


Overwriting app.py


In [3]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.5 MB/s eta 0:00:00


In [8]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.526s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [9]:
!streamlit run /content/app.py &>/content/logs.txt &

In [10]:
!npx localtunnel --port 8501

npx: installed 22 in 2.074s
your url is: https://proud-rules-draw.loca.lt
^C
